# Instalando o Spark

In [ ]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip


# Iniciar a sessão spark
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder                  
      .config('spark.ui.port', '4050')
      .appName("SparkSQL")
      .getOrCreate()
)

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8bbef8ff15f2fab7dc2673d9dbdba2534b283dbd7caa79d249260c2b339d67c0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://d056-34-125-176-38.ngrok.io


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# SparkML

Spark MLlib (abreviação de Spark Machine Learning library) é uma biblioteca em Apache Spark para fornecer funcionalidade de aprendizagem de máquinas. Ela fornece uma ampla gama de algoritmos para tarefas tais como classificação, regressão, clusterização, recomendação e extração de features, bem como ferramentas para construção, avaliação e ajuste de modelos de aprendizagem de máquinas.

Também fornece uma série de utilitários para trabalhar com dados, incluindo manuseio e pré-processamento de dados, extração de recursos e avaliação de modelos. Também possui APIs em Python, R e Java que permitem aos desenvolvedores usá-lo em uma variedade de linguagens de programação.

A Spark MLlib é construída sobre o principal motor de processamento de dados da Spark e é projetada para trabalhar com DataFrames e RDDs (Resilient Distributed Datasets) da Spark, para que possa aproveitar as capacidades de processamento distribuído da Spark para realizar tarefas de aprendizagem de máquinas em larga escala.


Para esse exemplo nós usaremos a regressão logística

- Regressão logística: A Regressão Logística é um algoritmo de aprendizado supervisionado que é comumente usado para tarefas de classificação binária. Pode ser útil para detectar fraudes porque pode modelar a relação entre as características de entrada e a probabilidade de uma transação ser fraudulenta.

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'drive/MyDrive/Colab Notebooks/case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)

ModuleNotFoundError: ignored

In [ ]:
df.printSchema()

df.show()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588

In [ ]:
from pyspark.sql.functions import col

df_flatten = df.withColumns({
    'destinatario_nome': col('destinatario').getField('nome'),
    'destinatario_banco': col('destinatario').getField('banco'),
    'destinatario_tipo': col('destinatario').getField('tipo'),
})

O StringIndexer é uma feature fornecido pela biblioteca MLlib do PySpark, usado para converter as strings em valores numéricos, que são mais adequados para algoritmos de aprendizagem de máquinas.

Ele funciona mapeando as strings para um conjunto fixo de índices, onde cada índice representa um valor de string único. 

Isto é útil quando se trabalha com variáveis categóricas, pois muitos algoritmos de aprendizagem de máquina exigem que as características de entrada sejam numéricas e não categóricas.

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCols=[
        "destinatario_nome", 
        "destinatario_banco",
        "destinatario_tipo",
        "categoria",
        "chave_pix"
    ], 
    outputCols=[
        "destinatario_nome_index", 
        "destinatario_banco_index",
        "destinatario_tipo_index",
        "categoria_index",
        "chave_pix_index"
    ])

df_index = indexer.fit(df_flatten).transform(df_flatten)
df_index.show()

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|        1000|            588.08|{Jonathan Gonsalv...|{Calebe Melo, Cai...|aleatoria|       outros|2021-07-16 05:00:55|     0|    

In [ ]:
cols_para_filtrar = [
  "valor",
  "transaction_date",
  "destinatario_nome_index", 
  "destinatario_banco_index",
  "destinatario_tipo_index",
  "chave_pix_index",
  "categoria_index",
  "fraude"
]

In [ ]:
is_fraud = df_index.select(cols_para_filtrar).filter("fraude == 1")
no_fraud = df_index.select(cols_para_filtrar).filter("fraude == 0")

Este código está usando o método sample() do DataFrame API do PySpark para selecionar aleatoriamente um subconjunto das linhas de um DataFrame chamado no_fraud. O método sample() é usado para selecionar aleatoriamente uma fração das linhas de um DataFrame.

O segundo argumento passado para o método da amostra() é 0,01, o que indica que apenas 1% das linhas do DataFrame devem ser selecionadas.

In [ ]:
no_fraud = no_fraud.sample(False, 0.01, seed = 123)

In [ ]:
df_concat = no_fraud.union(is_fraud)
df = df_concat.sort("transaction_date")
df.count()

16202

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 123)
print("train =", train.count(), " test =", test.count())

train = 11278  test = 4924


Em resumo, este código cria um UDF que pega os valores da coluna "fraude" e retorna uma variável binária (1 ou 0) dependendo se o valor é maior que 0 ou não. Em seguida, cria uma nova coluna "is_fraud" com estas variáveis binárias.

In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf

is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# Create the feature vectors.
# VectorAssembler is a transformer that combines a given list of columns into a single vector column.
assembler = VectorAssembler(
  inputCols = [x for x in train.columns if x not in ["transaction_date", "fraude", "is_fraud"]],
  outputCol = "features")

# Use Logistic Regression.
# is a machine learning algorithm that is used for classification tasks
lr = LogisticRegression().setParams(
    maxIter = 100000,
    labelCol = "is_fraud",
    predictionCol = "prediction")


# This will train a logistic regression model on the input data and return a 
# LogisticRegressionModel object which can be used to make predictions on new data.
model = Pipeline(stages = [assembler, lr]).fit(train)

In [ ]:
predicted = model.transform(test)

In [ ]:
predicted.show()

+-----+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+--------------------+----------+
|valor|   transaction_date|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|chave_pix_index|categoria_index|fraude|            features|       rawPrediction|         probability|prediction|
+-----+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+--------------------+----------+
| 0.18|2022-02-20 11:56:41|                 4288.0|                     4.0|                    1.0|            3.0|            5.0|     0|[0.18,4288.0,4.0,...|[200.715458857010...|           [1.0,0.0]|       0.0|
| 0.34|2021-12-10 12:59:50|                 7127.0|                     3.0|                    1.0|            0.0|            0.0|     0|[0.34

In [ ]:
predicted = predicted.withColumn("is_fraud", is_fraud(predicted.fraude))
predicted.crosstab("is_fraud", "prediction").show()

+-------------------+---+----+
|is_fraud_prediction|0.0| 1.0|
+-------------------+---+----+
|                1.0|  0|4660|
|                0.0|262|   2|
+-------------------+---+----+



In [ ]:
test.show()

+-----+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+
|valor|   transaction_date|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|chave_pix_index|categoria_index|fraude|
+-----+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+
| 0.18|2022-02-20 11:56:41|                 4288.0|                     4.0|                    1.0|            3.0|            5.0|     0|
| 0.34|2021-12-10 12:59:50|                 7127.0|                     3.0|                    1.0|            0.0|            0.0|     0|
| 0.49|2023-01-02 17:53:37|                 5426.0|                     3.0|                    1.0|            2.0|            6.0|     0|
| 0.51|2021-09-20 09:03:17|                10016.0|                     2.0|                    0.0|            0.0|            6.0|     0|
| 0.52|2022-03-29 06

In [ ]:
df_teste_cols = [
    'valor', 
    'transaction_date',
    'destinatario_nome_index',
    'destinatario_banco_index',
    'destinatario_tipo_index',
    'chave_pix_index',
    'categoria_index',
    'fraude'
  ]

df_teste_data = [
    (103.2, "2023-01-01 11:56:41", 328.0, 4.0, 1.0, 3.0, 5.0, 0),
    (500000.0, "2023-01-01 11:56:41", 328.0, 2.0, 3.0, 2.0, 5.0, 1),
    (19999.0, "2023-01-01 11:56:41", 328.0, 1.0, 2.0, 1.0, 5.0, 0),
]

In [ ]:
df_teste = spark.createDataFrame(df_teste_data).toDF(*df_teste_col)

In [ ]:
predicted = model.transform(df_teste)

In [ ]:
predicted.show()

+--------+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+-----------+----------+
|   valor|   transaction_date|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|chave_pix_index|categoria_index|fraude|            features|       rawPrediction|probability|prediction|
+--------+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+-----------+----------+
|   103.2|2023-01-01 11:56:41|                  328.0|                     4.0|                    1.0|            3.0|            5.0|     0|[103.2,328.0,4.0,...|[200.607483823920...|  [1.0,0.0]|       0.0|
|500000.0|2023-01-01 11:56:41|                  328.0|                     2.0|                    3.0|            2.0|            5.0|     1|[500000.0,328.0,2...|[-921

## Exercícios

- Regressão logística: A Regressão Logística é um algoritmo de aprendizado supervisionado que é comumente usado para tarefas de classificação binária. Pode ser útil para detectar fraudes porque pode modelar a relação entre as características de entrada e a probabilidade de uma transação ser fraudulenta.

- Random Forest: Random Forest é um algoritmo de aprendizado supervisionado que cria múltiplas árvores de decisão e depois combina os resultados. É comumente usado para tarefas de classificação e é conhecido por sua capacidade de lidar com grandes conjuntos de dados e dados de alta dimensão.

- Gradient-Boosted Treess: O Gradient-Boosted Trees é um método de conjunto que combina múltiplas árvores de decisão para melhorar a precisão do modelo. Também é comumente usado para tarefas de classificação e, como a Random Forest, pode lidar com grandes conjuntos de dados e dados de alta dimensão.

- SVM de uma classe: O SVM de uma classe é um algoritmo de aprendizado não supervisionado que pode ser usado para detectar aberrações ou anomalias nos dados, o que pode ser útil para detectar fraudes. Ele funciona através do aprendizado de um limite que separa a maioria dos dados dos outliers.

- Autoencoder: O Autoencoder é uma rede neural treinada para reconstruir sua entrada. Ele pode ser usado para detectar fraudes treinando-o com um conjunto de dados de transações normais e depois usando-o para detectar anomalias em novas transações.

- Isolation Forest: Isolation Forest é um algoritmo de aprendizagem não supervisionado que pode ser usado para detectar anomalias nos dados.